<a href="https://colab.research.google.com/github/MiguelAngel-ht/Machine_Learning_Models/blob/main/NLP_Intent_Matching_and_Transfer_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHAT BOT**

## *STEP 1*: INTENT MATCHING


In [ ]:
# 2 TYPES TO VECTORIZE A STRING
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# 2 TYPES TO MEASURE SIMILARITY BETWEEN 2 VECTORS
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# BASIC LIBRARIES TO VECTOR OPERATIONS AND TO GET DATE AND TIME
import numpy as np
from datetime import datetime

In [ ]:
# INITIALIZATION OF VECTORIZERS
tfid = TfidfVectorizer()
count = CountVectorizer(analyzer='word', ngram_range=(2, 2))

In [ ]:
def get_datetime():
  """ This function is to get current date and time"""
  return str(datetime.now())

In [ ]:
# DEFINITION OF LIST OF QUESTIONS AND ITS RESPECTIVE ANSWERS
Questions = ['what is this', 'Who is him', 'What time is it']
Answers = ['FAQ BOT CHIDO', 'He is John Wick', get_datetime()]

# INPUT TO TEST
input = ['what time is']
Questions.append(input[0])   # SAVE INPUT IN LAST POSITION

In [ ]:
def Similarity_TFID(Questions):
  """   Similarity of user's input and questions with vectorizer of TFID  
        
        atributes: list of strings with Questions       
        return: list of values with cosine similarity maching with every Question 
  """
  
  n = len(Questions)      # GET NUMBER OF QUESTIONS
  input_vectorized = tfid.fit_transform(Questions)    # OBJECT OF TFID VECTORIZER
  input_vectorized = input_vectorized.toarray()       # CASTING TO AN ARRAY [0.2, 0, 0.4, ...]
  best_match_index = []                               # VARIABLE WHEN SIMILARITIES WILL SAVED
  
  for i in input_vectorized[0:(n-1)]:
      """ Compute Similarity between user's input and all questions registered  """
      sim = cosine_similarity([i], [input_vectorized[-1]])[0][0]        # COMPUTE COSINE SIMILARITY
      best_match_index.append(sim)                                      # SAVE SIMILARITY
  
  return best_match_index         # RETURN AN ARRAY WITH ALL SIMILARITIES

In [ ]:
def Similarity_Counter(input_vectorized):
  """   Similarity of user's input and questions with vectorizer of COUNT  
        
        atributes: list of strings with Questions       
        return: list of values with jacard similarity maching with every Question 
  """
  
  n = len(Questions)          # GET NUMBER OF QUESTIONS
  input_vectorized = count.fit_transform(Questions)         # OBJECT OF COUNT VECTORIZER
  input_vectorized = input_vectorized.toarray()             # CASTING TO AN ARRAY [1, 0, 1, ...]
  best_match_index = []                                     # VARIABLE WHEN SIMILARITIES WILL SAVED

  for i in input_vectorized[0:(n-1)]:
      """Compute Similarity between user's input and all questions registered"""
      sim = jaccard_score(i, input_vectorized[-1])                # COMPUTE JACCARD SIMILARITY
      best_match_index.append(sim)                                # SAVE SIMILARITY

  return best_match_index       # RETURN AN ARRAY WITH ALL SIMILARITIES

In [ ]:
# GET SIMILARITIES AND ADD TO EACH OTHERS
best_match_index_a = Similarity_Counter(Questions)
best_match_index_b = Similarity_TFID(Questions)
best_match_index = np.add(best_match_index_a, best_match_index_b)

print('Best Match List: ', best_match_index)

# OBTAIN THE MAXIMUM VALUE AND INDEX OF IT
t = best_match_index.max()
i = np.argmax(best_match_index)

# IF T > TO AN UMBRAL 0.5, THE ANSWER IS ENOUGHTLY SIMILAR TO ANSWER
if t > 0.5:
  print('Answer: ', Answers[t])
else:
  print("""I don't Undertand""")

Best Match List:  [0.4597522  0.15836175 1.41864211]
Answer:  2022-11-04 19:52:07.802277


## *STEP 2*: TEXT CLASSIFICATION

In [ ]:
# @title Load Model and Functions

# IMPORTING MODEL
from tensorflow.keras.models import load_model
import pickle
import random
import nltk
# READ DATASET
import json
from google.colab import output

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = nltk.stem.WordNetLemmatizer()

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')

intents = json.loads(open('x.json').read())  # DIRECT DEFITION IN THE SAME CODE

def clean_up_Sentence(sentence):
  """ TOKENIZER INPUT DATA TO THE BOT """
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words


def bag_of_words(sentence):
  """ BINARIZATION OF INPUT SENTENCE """
  sentence_words = clean_up_Sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  output.clear()
  return np.array(bag)


def predict_class(sentence):
  """ PREDICTION OF THE MODEL FROM INPUT SENTENCE """
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_TRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r > ERROR_TRESHOLD]
  
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
  return return_list


def get_response(intents_list, intents_json):
  """ CHOICE RANDOM RESPONSE OF THE TRAINING RESPONSES DATA """
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  result = ''
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

output.clear()

In [76]:
# LOAD MODEL TO GET GREETING, GOODBYES AND THANKS

message = 'dcgkh'                       # INPUT 
ints = predict_class(message.lower())   # PREDICT CLASS (GREETING, GOODBYE, THANKS OR NOTHING)
res = get_response(ints, intents)       # GET RESPONSE
print("Bot: ",res)                      # PRINT RESPONSE

Bot:  I don't know what your mean :(


## *STEP 3*: TRANSFER LEARNING

In [73]:
# INSTALL LAST VERSION OF OPENAI 
!pip install --upgrade openai
output.clear()

In [ ]:
# IMPORT OPENAI
import openai

# API KEY TO USE IT
openai.api_key = "sk- YOUR KEY"

In [136]:
# INPUT 
input_ = 'whom is Lionel Messi'

# IF THE INPUT DOESN'T ENDS WITH "?", THEN IT ADD IT
if input[-1] != '?':
  input_ += '?'

# OPENAI API DAVINCI RECIVE A INPUT AND RETURN A JSON
response = openai.Completion.create(
  model = "text-davinci-002",
  prompt = input + " A:",
  temperature = 0,
  max_tokens = 100,
  top_p = 1,
  frequency_penalty = 0.0,
  presence_penalty = 0.0,
  stop=["\n"]
)

In [137]:
# PRINT RESPONSE OF API
print(response['choices'][0]['text'])

 Lionel Messi is an Argentine professional footballer who plays as a forward for Spanish club Barcelona and the Argentina national team.
